In [0]:
# REPLACE by your individual
my_individual = 'HG01149'

## Installing requirements

In [0]:
# Connect to github and load the necessary data and tools
%%capture
!export LD_LIBARY_PATH=/usr/lib/x86_64-linux-gnu:$LD_LIBRARY_PATH
!apt install libgsl-dev
!ln -s /usr/lib/x86_64-linux-gnu/libgsl.so /usr/lib/x86_64-linux-gnu/libgsl.so.0
!git clone https://github.com/priyamoorjani/CCB293.git
!chmod +x CCB293/bin/smartpca
!chmod +x CCB293/bin/admixture
!cd CCB293/data/1000G_archaic && unzip 1000G_archaic.geno.zip
!cd CCB293/data/1000G_archaic && git clone https://github.com/LauritsSkov/Introgression-detection.git

In [0]:
# SKIP
#%%capture
#!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/supporting/accessible_genome_masks/StrictMask/20140520.chr17.strict_mask.fasta.gz -O 20140520.chr17.strict_mask.fasta.gz
#!wget http://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/chromFaMasked.tar.gz
#!tar xzvf chromFaMasked.tar.gz
#!wget http://web.corral.tacc.utexas.edu/WGSAdownload/resources/human_ancestor_GRCh37_e71/homo_sapiens_ancestor_17.fa.gz
#!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel
#!apt install autoconf autogen
#!git clone https://github.com/vcftools/vcftools.git
#!cd vcftools && ./autogen.sh && ./configure && make && make install
#!git clone https://github.com/samtools/htslib.git
#!cd htslib && autoheader && autoconf && ./configure && make && make install
#!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr17.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz
#!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr17.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi
#!gunzip homo_sapiens_ancestor_17.fa.gz

In [0]:
# load the libraries
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import pandas as pd
import colorsys
import seaborn as sns
from IPython.display import Image
import os

In [0]:
# define plotting functions

def plot_pcs(pcs, I, J, labels, indivs=None):
    PCI = 'PC{}'.format(I)
    PCJ = 'PC{}'.format(J)
    plt.figure(figsize=(10,10))
    #colors = [colorsys.hsv_to_rgb(h,0.9,0.7) for h in np.linspace(0,1,len(np.unique(labels))+1)[:-1]]
    #colors = ['#acc2d9', '#653700', '#b2996e', '#a8ff04', 'xkcd:orange', '#894585',
    #          '#70b23f', '#d4ffff', '#65ab7c', '#952e8f', '#fcfc81', '#a5a391',
    #          '#388004', '#4c9085', '#5e9b8a', '#efb435', '#d99b82', '#0a5f38',
    #          '#0c06f7', '#61de2a', '#3778bf', '#2242c7', '#533cc6', '#9bb53c',
    #          '#05ffa6', '#1f6357', '#017374', '#0cb577']

    colors = [
        "#7e1e9c", "#15b01a", "#0343df", "#ff81c0","#653700","#e50000","#029386",
        "#f97306", "#96f97b", "#c20078", "#ffff14", "#95d0fc", "#929591",
        "#9a0eea",
        "#033500",
        "#00035b",
        "#06c2ac",
        "#d1b26f",
        "#00ffff",
        "#650021",
        "#ffb07c",
        "#ff796c",
        "#36013f",
        "#c65102",
        "#000000",
        "#c1f80a",
        "#4b5d16",
        "#0652ff"
    ]
    

    print(len(np.unique(labels)))
    for i, p in enumerate(np.unique(labels)):
        pcs_p = pcs[labels==p]
        plt.scatter(pcs_p[PCI], pcs_p[PCJ], label=p, color=colors[i])
    plt.legend(loc=(1.04,0))
    if indivs is not None:
        for individual in indivs:
            pcs_i = pcs[pcs.Sample==individual]
            plt.annotate(individual, [pcs_i[PCI], pcs_i[PCJ]])
            plt.plot(pcs_i[PCI], pcs_i[PCJ], 'o', color='black')
    plt.xlabel(PCI)
    plt.ylabel(PCJ);


def plot_admixture(admixture, population_indices, population_labels):
    plot = plt

    N,K = admixture.shape
    colors = [colorsys.hsv_to_rgb(h,0.9,0.7) for h in np.linspace(0,1,K+1)[:-1]]
    text_color = 'k'
    bg_color = 'w'
    fontsize = 24

    figure = plot.figure(figsize=(5,3))

    xmin = 0.13
    ymin = 0.2
    height = 1.2#0.6
    width = 1.5#0.74
    indiv_width = width/N
    subplot = figure.add_axes([xmin,ymin,width,height])
    [spine.set_linewidth(0.001) for spine in subplot.spines.values()]

    for k in np.arange(K):
        if k:
            bottoms = admixture[:,:k].sum(1)
        else:
            bottoms = np.zeros((N,),dtype=float)

        lefts = np.arange(N)*indiv_width
        subplot.bar(lefts, admixture[:,k], width=indiv_width, bottom=bottoms, facecolor=colors[k], edgecolor=colors[k], linewidth=0.4)

        subplot.axis([0, N*indiv_width, 0, 1])
        subplot.tick_params(axis='both', top=False, right=False, left=False, bottom=False)
        xtick_labels = tuple(map(str,['']*N))
        subplot.set_xticklabels(xtick_labels)
        ytick_labels = tuple(map(str,['']*K))
        subplot.set_yticklabels(ytick_labels)

    for p,popname in enumerate(population_labels):
        indices = np.where(population_indices==p)[0]
        if indices.size>0:
            vline_pos = (indices.max()+1)*indiv_width 
            subplot.axvline(vline_pos, linestyle='-', linewidth=0.2, c='#888888')
            label_position = (xmin+(2*indices.min()+indices.size)*0.5*indiv_width, ymin-0.01)
            figure.text(label_position[0], label_position[1], popname, fontsize=12, color='k', \
                horizontalalignment='right', verticalalignment='top', rotation=70)

    return figure

def plot_admix_individual(admix, individual):
    K = admix.shape[1]
    colors = [colorsys.hsv_to_rgb(h,0.9,0.7) for h in np.linspace(0,1,K+1)[:-1]]
    plt.pie(admix[individuals.individual==individual][0], colors=colors)
    plt.title(individual);

def plot_window_assignment(window_assignment):
    plt.scatter(np.arange(len(window_assignment)), window_assignment, s=8)
    plt.xlabel('window')
    plt.yticks([0,1], ['Human', 'Archaic'])

## Exploring the data

In [0]:
# set current directory
%cd /content/CCB293/data/1000G_archaic/

In [0]:
# Read the individual file. For more information about file formats, refer to: https://reich.hms.harvard.edu/software/InputFileFormats 
individuals = pd.read_csv('1000G_archaic.ind', delim_whitespace=True, header=None, names=['individual', 'sex', 'population'])

In [0]:
# 1000 Genomes Populations and 3 letter codes. For reference, see population_info.csv
%%writefile population_info.csv
population,description,super_population
CHB,Han Chinese in Beijing China,EAS
JPT,Japanese in Tokyo - Japan,EAS
CHS,Southern Han Chinese,EAS
CDX,Chinese Dai in Xishuangbanna - China,EAS
KHV,Kinh in Ho Chi Minh City - Vietnam,EAS
CEU,Utah Residents (CEPH) with Northern and Western European Ancestry,EUR
TSI,Toscani in Italia,EUR
FIN,Finnish in Finland,EUR
GBR,British in England and Scotland,EUR
IBS,Iberian Population in Spain,EUR
YRI,Yoruba in Ibadan - Nigeria,AFR
LWK,Luhya in Webuye - Kenya,AFR
GWD,Gambian in Western Divisions in the Gambia,AFR
MSL,Mende in Sierra Leone,AFR
ESN,Esan in Nigeria,AFR
ASW,Americans of African Ancestry in SW USA,AFR
ACB,African Caribbeans in Barbados,AFR
MXL,Mexican Ancestry from Los Angeles USA,AMR
PUR,Puerto Ricans from Puerto Rico,AMR
CLM,Colombians from Medellin - Colombia,AMR
PEL,Peruvians from Lima - Peru,AMR
GIH,Gujarati Indian from Houston - Texas,SAS
PJL,Punjabi from Lahore - Pakistan,SAS
BEB,Bengali from Bangladesh,SAS
STU,Sri Lankan Tamil from the UK,SAS
ITU,Indian Telugu from the UK,SAS

In [0]:
# Display population labels
population_info = pd.read_csv('population_info.csv')
population_info

In [0]:
# Update file with labels for archaic groups
individuals = individuals.merge(population_info, on='population', how='left').fillna('Archaic')

In [0]:
# 1000 Genomes super_population - AFR = Africa, EUR = Europe, SAS = South Asia, EAS = East Asia, AMR = Americas, Archaic = Neanderthal/ Denisova
individuals.population = individuals.population.astype('category')
individuals.super_population = pd.Categorical(
    individuals.super_population, 
    categories=['AFR', 'EUR', 'SAS', 'EAS', 'AMR', 'Archaic'],
    ordered=True)

In [0]:
# sort individuals by population
individuals = individuals.sort_values(['super_population', 'population'])

In [0]:
# Display Ind file. Format: https://reich.hms.harvard.edu/software/InputFileFormats
individuals.tail()

In [0]:
# order individual indexes
order = individuals.index

In [0]:
# Set my_individual's continental group
my_individual_index = np.where(individuals.individual==my_individual)[0][0]
my_continental_group = individuals.iloc[my_individual_index].super_population

In [0]:
# Display SNP file. Format: https://reich.hms.harvard.edu/software/InputFileFormats
snps = pd.read_csv('1000G_archaic.snp', delim_whitespace=True, header=None, names=['id', 'chr', 'recomb. rate', 'position', 'allele 1', 'allele 2'])
print(snps.shape)
snps.head()

In [0]:
# Display genotype file. Format: https://reich.hms.harvard.edu/software/InputFileFormats
genotypes = pd.read_fwf('1000G_archaic.geno', widths=np.ones(2506, dtype=int).tolist(), header=None)
print(genotypes.shape)
genotypes.head()

# Principal Component Analysis (PCA)

Patterson, Nick, Alkes L. Price, and David Reich. "Population structure and eigenanalysis." PLoS genetics 2.12 (2006): e190.

https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.0020190

Software documentation: https://github.com/DReichLab/EIG/tree/master/POPGEN

Usage: `!../../bin/smartpca -p 1000G_archaic.smartpca.par`

In [0]:
# Parameter file (1000G_archaic.smartpca.par)
%%writefile 1000G_archaic.smartpca.par
genotypename: 1000G_archaic.geno     # file with genotype information
snpname:      1000G_archaic.snp      # file with snp information
indivname:    1000G_archaic.ind      # file with individual information   
#poplistname:  1000G_archaic.pop.list # list of pops to include in the run.
outliermode: 2
evecoutname:  1000G_archaic.evec     # output file of eigenvectors.
evaloutname:  1000G_archaic.eval     # output file of all eigenvalues
snpweightoutname: 1000G_archaic.Weightout.txt
phylipname:   1000G_archaic.phyl     # file with Fst values across populations 
numoutevec:   20               # number of PCs to output 
numthreads:   1                # if running interactively, use 1 only

In [0]:
# Run smartpca 
%%capture
!../../bin/smartpca -p 1000G_archaic.smartpca.par

In [0]:
# Display results - evec file. File contains loadings for each individual on the 20 PCs. https://github.com/DReichLab/EIG/tree/master/POPGEN
n_pcs = 20
pcs = pd.read_csv('1000G_archaic.evec', skiprows=1, header=None, delim_whitespace=True,
                       names=['Sample']+['PC'+str(i+1) for i in range(n_pcs)]+['population'])
pcs = pcs.merge(population_info, on='population', how='left').fillna('Archaic')
pcs.head()

In [0]:
# Plot the top two PCs. Population labels = super_population
plot_pcs(pcs, 1, 2, pcs.super_population, [my_individual])

In [0]:
# Plot the top two PCs. Population labels = population
plot_pcs(pcs, 1, 2, pcs.population, [my_individual])

In [0]:
# Display eigenvalues. https://github.com/DReichLab/EIG/tree/master/POPGEN
eigenvalues = np.loadtxt('1000G_archaic.eval')
plt.plot(np.arange(1, 21), eigenvalues[:20])
plt.xticks(np.arange(1, 21));
plt.xlabel('eigenvalue rank')
plt.ylabel('eigenvalue');

In [0]:
# Display %variance explained
eigenvalues = np.loadtxt('1000G_archaic.eval')
plt.plot(np.arange(1, 21), 100 * eigenvalues[:20] / eigenvalues.sum())
plt.xticks(np.arange(1, 21));
plt.xlabel('eigenvalue rank')
plt.ylabel('percent variance explained');

In [0]:
# FST between populations
!cat 1000G_archaic.phyl

## Exercise: explore your continental group

In [0]:
# Write population list. 
np.savetxt('pop.list', individuals[individuals.super_population==my_continental_group].population.unique().astype(str), fmt='%s')

In [0]:
%%writefile 1000G_archaic.continent.smartpca.par
genotypename: 1000G_archaic.geno     # file with genotype information
snpname:      1000G_archaic.snp      # file with snp information
indivname:    1000G_archaic.ind      # file with individual information   
poplistname:  pop.list # list of pops to include in the run.
evecoutname:  1000G_archaic.continent.evec     # output file of eigenvectors.
evaloutname:  1000G_archaic.continent.eval     # output file of all eigenvalues
snpweightoutname: 1000G_archaic.continent.Weightout.txt
phylipname:   1000G_archaic.continent.phyl     # file with Fst values across populations 
numoutevec:   10               # number of PCs to output 
numthreads:   1                # if running interactively, use 1 only
outliermode: 2
#outliermode should be 0, 1 or 2 
#mode = 2  NO outlier removal 
#mode = 1  when calculating mean and standard deviation of a PC to decide whether to remove a sample the 
# sample itself is not used.   This may be important for datasets with very small sample sizes (say less than 30).  
#mode = 0  (default) use all samples to compute PC mean and variance. 

In [0]:
# Run smartpca
%%capture
!../../bin/smartpca -p 1000G_archaic.continent.smartpca.par

In [0]:
# Display output
n_pcs = 10
pcs = pd.read_csv('1000G_archaic.continent.evec', skiprows=1, header=None, delim_whitespace=True,
                       names=['Sample']+['PC'+str(i+1) for i in range(n_pcs)]+['population'])
pcs = pcs.merge(population_info, on='population', how='left').fillna('Archaic')
pcs = pcs[pcs.super_population==my_continental_group]
pcs.head()

In [0]:
# plot PCA output
plot_pcs(pcs, 1, 2, pcs.population, [my_individual])

In [0]:
# Display information about current individual from 1000G
individuals.iloc[my_individual_index]

In [0]:
# FST between populations
!cat 1000G_archaic.continent.phyl

# Admixture analysis

Alexander, David H., John Novembre, and Kenneth Lange. "Fast model-based estimation of ancestry in unrelated individuals." Genome research 19.9 (2009): 1655-1664.

https://genome.cshlp.org/content/19/9/1655.full

Software documentation: http://www.genetics.ucla.edu/software/admixture/admixture-manual.pdf

Usage: 

```!../../bin/admixture --cv 1000G_archaic.bed K```

where K = number of clusters

In [0]:
# Run ADMIXTURE with K=2 clusters
!../../bin/admixture --cv 1000G_archaic.bed 2

In [0]:
# Display output of ADMIXTURE. Each line contains the probability of an individual belonging to each cluster - not be to confused with ancestry proportion.
admix = pd.read_csv('1000G_archaic.2.Q', ' ', header=None).values[order]
admix

In [0]:
# Admixture plot where each color represents one cluster
plot_admixture(admix, individuals.super_population.cat.codes, individuals.super_population.cat.categories);

In [0]:
# Admixture plot with subcontinental ancestry where each color represents one cluster
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

In [0]:
# Admixture output for the current individual
plot_admix_individual(admix, my_individual)

In [0]:
# Admixture output for Altai Neanderthal. 
plot_admix_individual(admix, 'Altai')

In [0]:
# Admixture output for Denisova. 
plot_admix_individual(admix, 'Denisova')

In [0]:
# Now let's run admixture with K=3 clusters
!../../bin/admixture --cv 1000G_archaic.bed 3

In [0]:
# Display output of ADMIXTURE (K=3)
admix = pd.read_csv('1000G_archaic.3.Q', ' ', header=None).values[order]
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

In [0]:
# Admixture output for the current individual (K=3)
plot_admix_individual(admix, my_individual)

In [0]:
# For K=4,5,6 results have been precomputed, we can just load them. Display output of ADMIXTURE (K=4)
admix = pd.read_csv('1000G_archaic.4.Q', ' ', header=None).values[order]
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

In [0]:
# Admixture output for the current individual (K=4)
plot_admix_individual(admix, my_individual)

In [0]:
# Display output of ADMIXTURE (K=5)
admix = pd.read_csv('1000G_archaic.5.Q', ' ', header=None).values[order]
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

In [0]:
# Admixture output for the current individual (K=5)
plot_admix_individual(admix, my_individual)

In [0]:
# Display output of ADMIXTURE (K=6)
admix = pd.read_csv('1000G_archaic.6.Q', ' ', header=None).values[order]
plot_admixture(admix, individuals.population.cat.codes, individuals.population.cat.categories);

In [0]:
# Admixture output for the current individual (K=6)
plot_admix_individual(admix, my_individual)

In [0]:
# Likelihood of data as a function of K, the number of clusters
K = np.arange(2, 9)
log_likelihood = [-40488756, -39516004, -39261878, -39067307, -39034320, -38999856, -38959044]
plt.plot(K, log_likelihood)
plt.xticks(K)
plt.xlabel('K')
plt.ylabel('log likelihood');

# Analysis of archaic introgression

Skov, Laurits, et al. "Detecting archaic introgression using an unadmixed outgroup." PLoS genetics 14.9 (2018): e1007641.

https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1007641

Software: https://github.com/LauritsSkov/Introgression-detection

Usage:

`!python2 Introgression-detection/Train.py infile outprefix model weights_file mutfile`

`!python2 Introgression-detection/Decode.py infile outprefix model weights_file mutfile window_size`

In [0]:
# Signal of archaic admixture
Image(filename='hmm-skov-3.png', height=500)

In [0]:
# What is an HMM
Image(filename='hmm.png', height=200) 

In [0]:
# Main idea for identifying segments of archaic ancestry
Image(filename='hmm-skov-2.png', height=500)

In [0]:
# SKIP
#!python2 Introgression-detection/MakeMaskfiles.py chr17.fa.masked 20140520.chr17.strict_mask.fasta.gz 1000 17 chr17
#!cp chr17.txt weights.txt
#!cp chr17.bed weights.bed
#!cat integrated_call_samples_v3.20130502.ALL.panel | grep -E "YRI|ESN|MSL" | cut -f 1 > outgroups.txt
#!tabix -h ALL.chr17.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz -R chr17.bed | vcftools --vcf - --counts --stdout --keep outgroups.txt --remove-indels --min-alleles 2 --max-alleles 2 > chr17.freq
#!python2 Introgression-detection/Estimate_mutationrate.py chr17.freq 1000000 1000 chr17.txt chr17.mut
#!cp chr17.mut mutationrates.txt
#!tabix -fh ALL.chr17.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz -R chr17.bed | vcftools --vcf - --indv HG00096 --remove-indels --min-alleles 2 --max-alleles 2 --stdout --counts | python2 Introgression-detection/Filtervariants.py homo_sapiens_ancestor_17.fa chr17.freq 1000 chr17.txt HG00096.chr17.observations.txt

In [0]:
# Configure files for the current individual
os.system('cp {}.chr17.observations.txt observations.txt'.format(my_individual));

In [0]:
# Define initial parameters for the HMM
%%writefile StartingParameters.hmm
# State names (only used for decoding)
states = ['Human','Archaic']

# Initialization parameters (prob of staring in states)
starting_probabilities = [0.98, 0.02]

# transition matrix
transitions = [[0.9995,0.0005],[0.012,0.98]]

# emission matrix (poisson parameter)
emissions = [0.04, 0.1]

In [0]:
# Train the HMM model
!python2 Introgression-detection/Train.py observations.txt trained StartingParameters.hmm weights.txt mutationrates.txt

In [0]:
# Display trained parameters
!cat trained.hmm

In [0]:
# Decode the most likely assignment of ancestry to windows
!python2 Introgression-detection/Decode.py observations.txt decoded trained.hmm weights.txt mutationrates.txt 1000

In [0]:
# Inference for every 1000Kb window
posterior_probs = pd.read_csv('decoded.All_posterior_probs.txt', '\t')
posterior_probs.head()

In [0]:
posterior_probs[posterior_probs.observations>0].head()

In [0]:
# The most likeliy assignment of ancestry to the whole chromosome, split into contiguous segments of same ancestry
decoded_summary = pd.read_csv('decoded.Summary.txt', '\t')
decoded_summary.head()

In [0]:
# Obtain the windows ancestry assignment
window_assignment = (posterior_probs.Mostlikely=='Archaic')

In [0]:
# Plot the assignment of the whole chromosome
plot_window_assignment(window_assignment)

In [0]:
# Plot assignment of the first 5000 windows
plot_window_assignment(window_assignment[:5000])

In [0]:
# Obtain number of observations per window of Human or Archaic ancestry
obs_human = posterior_probs.observations[posterior_probs.Mostlikely=='Human']
obs_archaic = posterior_probs.observations[posterior_probs.Mostlikely=='Archaic']
max_human = obs_human.max()
max_archaic = obs_archaic.max()

In [0]:
# Plot the number of variants per window of human ancestry
plt.bar(np.arange(max_human+1), [(obs_human==i).mean() for i in range(max_human+1)]);
plt.xticks(np.arange(max_human+1));
plt.xlabel('Number of variants')
plt.ylabel('Proportion')
plt.title('Histogram of number of variants per window of Human ancestry');

In [0]:
# Plot the number of variants per window of archaic ancestry
plt.bar(np.arange(max_archaic+1), [(obs_archaic==i).mean() for i in range(max_archaic+1)]);
plt.xticks(np.arange(max_archaic+1));
plt.xlabel('Number of variants')
plt.ylabel('Proportion')
plt.title('Histogram of number of variants per window of Archaic ancestry');